In [ ]:
#I WROTE ARTICLE ONE AND TWO CODES TOGETHER
import os
import requests
from bs4 import BeautifulSoup
import json
import time
from confluent_kafka import Producer, KafkaException

# Kafka yapılandırması
kafka_conf = {
    'bootstrap.servers': 'localhost:9092'
}
producer = Producer(kafka_conf)
topic = 'urun_verileri'

# Web sitesi URL'si
url = "https://scrapeme.live/shop/"

# Verilerin kaydedileceği dosya yolu
file_path = 'urun_verileri.json'

# URL'ye GET isteği gönder
response = requests.get(url)
print("Ana sayfa isteği gönderildi.")

# İçeriği BeautifulSoup ile parse et
soup = BeautifulSoup(response.content, 'html.parser')
print("Ana sayfa içeriği parse edildi.")

# Sayfadaki tüm ürünleri bul
urunler = soup.find_all('li', class_='product')
print(f"{len(urunler)} ürün bulundu.")

def save_to_file(data_list):
    # Dosya mevcutsa içeriğini oku, değilse yeni bir liste oluştur
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            existing_data = json.load(file)
        existing_data.extend(data_list)
    else:
        existing_data = data_list

    # Verileri dosyaya yaz
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False, indent=4)

    print(f"Veriler dosyaya kaydedildi: {file_path}")

# Her ürün için gerekli verileri çıkar
data_list = []
for urun in urunler:
    urun_adi = urun.find('h2', class_='woocommerce-loop-product__title').text
    urun_fiyati = urun.find('span', class_='woocommerce-Price-amount amount').text
    urun_linki = urun.find('a')['href']

    # Temel bilgileri yazdır
    print(f"Ürün Adı: {urun_adi}")
    print(f"Ürün Fiyatı: {urun_fiyati}")
    print(f"Ürün Linki: {urun_linki}")
 
    # Ürün detay sayfasını ziyaret et
    urun_response = requests.get(urun_linki, timeout=10)
    urun_soup = BeautifulSoup(urun_response.content, 'html.parser')
 
    urun_aciklamasi = urun_soup.find('div', class_='woocommerce-product-details__short-description')
    urun_aciklamasi_text = urun_aciklamasi.text.strip() if urun_aciklamasi else 'Açıklama yok'
 
    urun_stok = urun_soup.find('p', class_='stock')
    urun_stok_text = urun_stok.text if urun_stok else 'Stok bilgisi yok'
 
    # Ek bilgileri yazdır
    print(f"Ürün Açıklaması: {urun_aciklamasi_text}")
    print(f"Ürün Stok Durumu: {urun_stok_text}")
    print("-" * 50)
 
    # Veriyi JSON formatına dönüştür ve Kafka'ya gönder
    urun_verisi = {
        'name': urun_adi,
        'price': urun_fiyati,
        'description': urun_aciklamasi_text,
        'stock': urun_stok_text
    }

    try:
        producer.produce(topic, key=urun_adi, value=json.dumps(urun_verisi))
        print(f"{urun_adi} Kafka'ya gönderildi.")
    except KafkaException as e:
        print(f"Hata: {e}")
 
    # Dosyaya kaydetmek için veriyi listeye ekle
    data_list.append(urun_verisi)

    # 1 saniye bekle
    time.sleep(1)

# Flush işlemi ile mesajları Kafka'ya gönder
try:
    print("Mesajlar Kafka'ya gönderiliyor...")
    producer.flush(10)
    print("Mesajlar başarıyla gönderildi.")
except KafkaException as e:
    print(f"Hata: {e}")

# Verileri dosyaya kaydet
save_to_file(data_list)

print("İşlem tamamlandı.")



In [ ]:
#I LEFT THE SECOND ARTICLE AND THE 3RD ARTICLE HERE BECAUSE I COULD NOT CONNECT DOCKER AND PYHTON AND COULD NOT SOLVE THE ERROR
from confluent_kafka import Consumer, KafkaException, KafkaError
import json
import os

# Kafka Consumer yapılandırması
kafka_conf = {
    'bootstrap.servers': 'localhost:9092',  # Kafka broker'in adresi ve portu
    'group.id': 'my_group',  # Consumer group id'si
    'auto.offset.reset': 'earliest'
}
consumer = Consumer(kafka_conf)
topic = 'urun_verileri'

def consume_and_save():
    print("Kafka'dan veri tüketme işlemine başlanıyor...")
    consumer.subscribe([topic])
    print(f"{topic} topic'ine abone olundu.")

    data_list = []

    try:
        while True:
            print("Veri bekleniyor...")
            msg = consumer.poll(1.0)
            if msg is None:
                print("Mesaj yok, tekrar bekleniyor...")
                continue
            if msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    print("Partition sonu.")
                    continue
                else:
                    print(f"Hata: {msg.error()}")
                    break

            # Kafka mesajını al ve JSON formatına dönüştür
            data = json.loads(msg.value().decode('utf-8'))
            data_list.append(data)
            print(f"Alınan veri: {data}")

            # Belirli sayıda veri toplandıktan sonra dosyaya kaydet
            if len(data_list) >= 10:
                save_to_file(data_list)
                data_list = []

    except KeyboardInterrupt:
        pass
    finally:
        # Kapatma işlemleri
        print("Consumer kapatılıyor...")
        consumer.close()
        if data_list:
            save_to_file(data_list)

def save_to_file(data_list):
    print("Veriler dosyaya kaydediliyor...")
    # Dosya adı ve yolu
    file_path = "urun_verileri.json"
    
    # Dosya mevcutsa içeriğini oku, değilse yeni bir liste oluştur
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            existing_data = json.load(file)
        existing_data.extend(data_list)
    else:
        existing_data = data_list
    
    # Verileri dosyaya yaz
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False, indent=4)

    print(f"Veriler dosyaya kaydedildi: {file_path}")

# Fonksiyonu çalıştır
consume_and_save()
